# Amazon Fine Food Reviews Analysis
Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon .
1. Number of reviews: 568,454
2. Number of users: 256,059
3. Number of products: 74,258
4. Timespan: Oct 1999 - Oct 2012
5. Number of Attributes/Featurs/column in data: 10



 Attribute/Features Information:

1. Id: Row Id
2. ProductId: Unique identifier for the product    (74258 =unique values)
3. UserId: Unqiue identifier for the user (256059=unique values)
4. ProfileName: Profile name of the user (218418=unique values)
5. HelpfulnessNumerator :Number of users who found the review helpful
6. HelpfulnessDenominator :Number of users who indicated whether they found the review helpful or not
7. Score: Rating between 1 and 5
8. Time: Timestamp for the review


Objective: Given a new review determine/predict whether the review is positive or negative


Q) How to detemine if a review is positve or negative?

Answer: We could use the Rating/Score.A rating of 4 0r 5 could be considered as positve and a rating of 1 or 2 could be considered as negative. A rating of 3 could be considered as neutral and hence could be ignored . This is approximate or proxy way of determining the polarity(negative and positive) of a review.


# Loading the Data 
The dataset is loaded in two forms:
1. .csv file
2. SQlitedatabase

In order to load the data, we are using the SQLITEdataset as it easier to query the data and visualise it efficiently .

Here as we only want to get global sentiment of the reviews (positive or negative), we will purposefully  ignore  all the Score equal to 3 .If the score is above 3 ,then the review will be "positive".Otherwise ,it will be set to "negative"

In [21]:
# import important library
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np
import string 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import stopwords

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import re
import pickle
from tqdm import tqdm
import os


In [22]:
#using Sqlite table to read data 
con=sqlite3.connect('database.sqlite')

#filtering only +ve and -ve review & not taking into consideration those review whose score =3
filtered_data=pd.read_sql_query(""" SELECT * FROM Reviews Where Score !=3""",con)

def partition(x):  #defining a fucntion for seperating positive and negative review
    if x<3:
        return 0
    return 1
#changing reviews with the score less than 3 to be negative & score more than 3 to be positive
actual_score=filtered_data['Score']
positive_negative=actual_score.map(partition)
# it will use the partion func we have created and chang the score in 1 or 0and 
#save in variable "Positive_negative"

filtered_data['Score']=positive_negative # it will replace  the score column with 0 and 1 (from 1,2&4,5 to 0 &1)
print('No of data points and column in our dataset:',filtered_data.shape)
filtered_data.head(3)

No of data points and column in our dataset: (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [23]:
#Counting the no. of time the same user has given the review 
display= pd.read_sql_query("""
SELECT UserId,ProductId ,ProfileName,Time,Score,Text,COUNT(*)
FROM Reviews 
GROUP BY UserId
HAVING COUNT(*)>1""",con)
display.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [24]:
display.shape

(80668, 7)

In [25]:
display['COUNT(*)'].sum() #total no. of distince user give the reviews 

393063

# Exploratory Data Analysis

### Data Deduplication:
It is observed that reviews has many deplicate entries.Hence it was neccessary toh remove duplicates entries inorder to get the unbaised results for the analysis of data.

In [26]:
display=pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score !=3 AND Userid="AR5J8UI46CURR" 
ORDER BY ProductId""",con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that

ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies,8.82-Ounce Packages (Pack of 8)

ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence inorder to reduce reduntancy it was decided to eleminate the rows having same parameters.

The method used for the same was that we first sort the data according to ProductId and then just keep the first similiar product reviews and delete the others .This method ensures that there is only one representatives for each product and deduplication without sorting would lead to possibility of different reprsentatives still existing for the same product.


In [27]:
# Sorting data according to ProductId in acsending order
sorted_data= filtered_data.sort_values('ProductId',axis=0,ascending=True,inplace=False,kind='quicksort',na_position='last')
sorted_data.shape

(525814, 10)

In [28]:
final_data=sorted_data.drop_duplicates(subset={"UserId","ProfileName" , "Time","Text"},keep='first',inplace=False)
final_data.shape

(364173, 10)

In [29]:
#Checking how much % of data stil remains:
(final_data['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

<b> Observation </b>: It has been seen that in few rows given below the value of  HelpfulnessNumerator is greater than HelpfulnessDenominator which is practically impossible hence these rows are removed from the calculation.

In [30]:
display=pd.read_sql_query("""SELECT * From Reviews Where Score !=3 
AND Id=44737 or Id=64422 ORDER BY ProductId
""" ,con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [31]:
#Keeping only that data whose HelpfulnessNumerator<=HelpfulnessNumerator
final_data=final_data[final_data['HelpfulnessNumerator']<=final_data['HelpfulnessNumerator']]

In [32]:
#How many positive and negative reviews are present in the dataset ?
print('There are',final_data['Score'].value_counts()[1],'postive points and' ,final_data['Score'].value_counts()[0], 'negative points')

There are 307063 postive points and 57110 negative points


# Text Preprocessing.
Till we have finished data deduplication ,now we will to preprocessing part before we go for further analysis and making the prediction models .

Hence in Preprocessing part we will do the the following things:

1. Removing of HTML tags.
2. Removing any punctuation or spme special character like , or . or # etc.
3. Check the word is made of made up of english letter and is not alpha-numeric
4. Check to see if the length of the word is greater than 2(as it was research that there is no adjective in 2 letters).
5. Convert the words to lowercase .
6. Remove the stopwords
7. Using Snowball Stemming the word (now using Porter Stemming as Snowball Stemming is better than Porter Stemming).

In [33]:
#finding the html tags in sentences
i=0 
for review in final_data['Text']:
    if(len(re.findall('<.*?>',review))):
        print(i)
        print(review)
        break
    i+=1

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [34]:
import re 
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop_words =set(stopwords.words('english')) # creating set of stopswords
print(stop_words) #it will show all the stopwords in NLTK
exclding_stop=['against','not','don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
             'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
             'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",'shouldn', "shouldn't", 'wasn',
             "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
print('****'*15)
stop_words=[word for word in stop_words if word not in exclding_stop]
print(' ')
print(stop_words)

snow_stem=nltk.stem.SnowballStemmer('english') #intialising the snowball stemmer
print(' ')
print('****'*15)
print('Base word of tasty:',snow_stem.stem('tasty')) # it will tell us the base word or do stemming

# creating a function to clean the word of any html-tags. The function will remove the html tag and evrything between them with "1" space 
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext
def cleanpunc(sentence):
    clean=re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r'',clean)
    return cleaned

{'our', 'until', "didn't", 'no', 'should', "you're", 'your', "won't", 't', 'down', "that'll", 'both', 'any', 'm', 'itself', 'while', "isn't", 'too', 'his', 'mustn', 'doesn', 'further', 'again', 'but', 'don', 'is', 'o', 'i', "hasn't", 'the', 'from', 'where', 'before', 'such', 'a', 'are', 'having', 'under', 'ain', 'at', 'their', 'by', 'hadn', 'or', 'there', 'which', 'been', 'all', 'whom', 'it', "needn't", 'how', "you've", 'we', "mightn't", 'between', 'below', 'with', 'than', "should've", 'weren', 'an', 'd', 're', 'her', 'few', 'herself', 'him', "couldn't", 'yours', 'myself', "you'd", 'shouldn', 'on', 'yourself', 'couldn', "haven't", 'once', 'had', "aren't", "she's", 'only', 'its', 'that', 'them', 'were', 'more', 'why', "hadn't", 'each', 'she', 'hers', 'isn', 'did', 'after', 'some', 'be', 'those', 'theirs', 'do', 'who', 'same', 'aren', 'will', 'to', "you'll", 'won', "don't", 'over', 'my', 'most', 'in', 'shan', 'about', 'now', 'ma', 'they', 'mightn', 'yourselves', 'into', 'against', 's', '

In [40]:
import os 
if not os.path.isfile('final1.sqlite'):
    final_string=[]
    all_positive_words=[]   # storing all the words from +ve review .
    all_negative_words=[]   # storing all the words from -ve review .
    for i ,review in enumerate(tqdm(final_data['Text'].values)):
        filter_sentences=[]
        review=cleanhtml(review)
        for word in review.split():
            #Using cleanpunc(w).split(),, one more split function here example:w="abc.def" ,cleanpunc will return "abc def"
            #if we dont use .split() function then we will be considering "abc def" as a single word, 
            #but if we use .split() function we will get "abc",def
            for clean_words in cleanpunc(word).split():
                if((clean_words.isalpha())&(len(clean_words)>2)):
                    if(clean_words.lower() not in stop_words):
                        s=(snow_stem.stem(clean_words.lower())).encode('utf-8')
                        filter_sentences.append(s)
                        if (final_data['Score'].values[i])==1:
                            all_positive_words.append(s)  #list of all words used to describe positive reviews
                        if (final_data['Score'].values[i])==0:
                            all_negative_words.append(s) #list of all words used to describe negative reviews 
                    else :
                        continue
                else:
                    continue
        str1=b" ".join(filter_sentences) # final cleaned words
        final_string.append(str1)
        i+=1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364173/364173 [17:40<00:00, 343.55it/s]


In [36]:
# https://stackoverflow.com/a/47091490/4084039
import re
from bs4 import BeautifulSoup
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [41]:
#adding column of Clean_text which display the data after pre-processing of the review.
final_data['Cleaned_text']=final_string 
final_data['Cleaned_text']=final_data['Cleaned_text'].str.decode('utf-8')

In [42]:
final_data['Time']=pd.to_datetime(final_data['Time'],unit='s')
#final_data

# Sorting dataset based on 'Time' feature

In [43]:
final_data=final_data.sort_values(by="Time")
#final_data.head(10)

In [44]:
# Store the final table into an SQLITE table for future
con=sqlite3.connect('final1.sqlite')
c=con.cursor()
con.text_factory=str
final_data.to_sql('reviews',con,schema=None,if_exists='replace',index=True,index_label=None,chunksize=None,dtype=None)
con.close()

1. Apply Knn(brute force version) on these feature sets
 * Review text, preprocessed one converted into vectors using (BOW)
 * Review text, preprocessed one converted into vectors using (TFIDF)
 * Review text, preprocessed one converted into vectors using (AVG W2v)

2. Apply Knn(kd tree version) on these feature sets:    
sklearn implementation of kd-tree accepts only dense matrices, you need to convert the sparse matrices of CountVectorizer/TfidfVectorizer into dense matices. You can convert sparse matrices to dense using .toarray() attribute. 
    * Review text, preprocessed one converted into vectors using (BOW) but with restriction on maximum features generated.
            count_vect = CountVectorizer(min_df=10, max_features=500) 
            count_vect.fit(preprocessed_reviews)
            
    * Review text, preprocessed one converted into vectors using (TFIDF) but with restriction on maximum features generated.
                tf_idf_vect = TfidfVectorizer(min_df=10, max_features=500)
                tf_idf_vect.fit(preprocessed_reviews)
            
    * Review text, preprocessed one converted into vectors using (AVG W2v)
    * Review text, preprocessed one converted into vectors using (TFIDF W2v)

3. The hyper paramter tuning(find best K)
    * Find the best hyper parameter which will give the maximum AUC value
    * Find the best hyper paramter using k-fold cross validation or simple cross validation data
    * Use gridsearch cv or randomsearch cv or you can also write your own for loops to do this task of hyperparameter tuning

4. Representation of results
    * You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    * Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    * Along with plotting ROC curve, you need to print the confusion matrix with predicted and original labels of test data points
5. Conclusion
    * You need to summarize the results at the end of the notebook, summarize it in the table format. To print out a table please refer to this prettytable library link



1. There will be an issue of data-leakage if you vectorize the entire data and then split it into train/cv/test.
2. To avoid the issue of data-leakag, make sure to split your data first and then vectorize it.
3. While vectorizing your data, apply the method fit_transform() on you train data, and apply the method transform() on cv/test data.

# 1. Applying KNN brute force on BOW

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [46]:
# spliting data into train,cross_val,test
import warnings
from sklearn.model_selection import train_test_split
X=final_data['Cleaned_text']
y=final_data['Score']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.30, random_state=0)
X_tr, X_cv, Y_tr, Y_cv = train_test_split(X_train, Y_train, test_size=.30, random_state=0)
                                          
                                          
print('X_train, Y_train:', X_train.shape, Y_train.shape)
print('X_test, Y_test:', X_test.shape, Y_test.shape)
print('X_tr, Y_tr:', X_tr.shape, Y_tr.shape)
print('X_cv, Y_cv:', X_cv.shape, Y_cv.shape)

X_train, Y_train: (254921,) (254921,)
X_test, Y_test: (109252,) (109252,)
X_tr, Y_tr: (178444,) (178444,)
X_cv, Y_cv: (76477,) (76477,)


## Converting text into vectors using BoW

In [47]:
count_vect = CountVectorizer() #in scikit-learn
X_train_bow = count_vect.fit_transform(X_tr)
print("The type of count vectorizer ",type(X_train_bow))
print("The shape of out text BOW vectorizer ",X_train_bow.get_shape())
#print("the number of unique words ", final_counts.get_shape()[1])
X_cv_bow = count_vect.fit_transform(X_cv)
X_test_bow = count_vect.fit_transform(X_test)
print("CV Data Size: ",X_cv_bow.shape)
print("Test Data Size: ",X_test_bow.shape)

The type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
The shape of out text BOW vectorizer  (178444, 77231)
CV Data Size:  (76477, 46348)
Test Data Size:  (109252, 57245)


In [48]:
#Normalize Data
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
X_train_bow=preprocessing.normalize(X_train_bow)
X_cv_bow=preprocessing.normalize(X_cv_bow)
X_test_bow=preprocessing.normalize(X_test_bow)
print("The shape of out text BOW vectorizer ",X_train_bow.get_shape())
print("CV Data Size: ",X_cv_bow.shape)
print("Test Data Size: ",X_test_bow.shape)

The shape of out text BOW vectorizer  (178444, 77231)
CV Data Size:  (76477, 46348)
Test Data Size:  (109252, 57245)


# Brute Force with Accuracy as metric:

In [49]:
mylist= list(range(0,50))
neighbours=list(filter(lambda x: x%2!=0,mylist))
# empty list that will hold cv scores
cv_scores = []
for k in neighbours:
    knn= KNeighborsClassifier(n_neighbors=k,algorithm='brute')
    scores=cross_val_score(knn,X_train_bow,Y_tr,cv=10,scoring='accuracy')
    cv_scores.append(scores.mean())

KeyboardInterrupt: 

In [ ]:
# changing to misclassification error
MSE=[1-x for x in cv_scores]
#determining optimal K
optimal_k=neighbours[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)